## Error Analysis

### Nesting Level

In [3]:
import re

# Function to calculate the nesting level of a query
def nesting_level(query):
    query = re.sub(r'\s+', '', query)  # Remove whitespace
    max_nesting = 0
    current_nesting = 0

    for char in query:
        if char == '{':
            current_nesting += 1
            max_nesting = max(max_nesting, current_nesting)
        elif char == '}':
            current_nesting -= 1

    return max_nesting

In [34]:
# Function that calculate the nesting level for each query and add it as a new column
dev['nesting_level'] = dev['query'].apply(nesting_level)

test = True
stats = False

if test:
    query1 = 'query{teacher(where:{_or:[{age:{_eq:\"32\"}},{age:{_eq:\"33\"}}]}){name}}'
    print(nesting_level(query1))
    query2 = 'query { templates ( where : { _and : [ { template_type_code : { _neq : "PP" } } , { template_type_code : { _neq : "PPT" } } ] } ) { template_id template_type_code } }'
    print(nesting_level(query2))
    query3 = 'query { countrylanguage ( where : { language : { _neq : \"French\" } } , distinct_on : countrycode ) { countrycode } }'
    print(nesting_level(query3))
    query4 = 'query { matches_aggregate { aggregate { count } } }'
    print(nesting_level(query4))
    query5 = 'query{singer_aggregate(where:{country:{_eq:\"France\"}}){aggregate{avg{age}min{age}max{age}}}}'
    print(nesting_level(query5))

if stats:
    # calculate the mean, median, min, and max of nesting levels
    mean_nesting = dev['nesting_level'].mean()
    median_nesting = dev['nesting_level'].median()
    min_nesting = dev['nesting_level'].min()
    max_nesting = dev['nesting_level'].max()

    print(f"Mean nesting level: {mean_nesting}")
    print(f"Median nesting level: {median_nesting}")
    print(f"Minimum nesting level: {min_nesting}")
    print(f"Maximum nesting level: {max_nesting}")

    # plot a histogram of the nesting levels
    plt.hist(dev['nesting_level'], edgecolor='black')
    plt.xlabel('Nesting Level')
    plt.ylabel('Frequency')
    plt.title('Histogram of Nesting Levels')
    plt.show()

    dev[dev['nesting_level'] == 2].shape[0]
    print("Nesting level 2 " + str(dev[dev['nesting_level'] == 2].shape[0]))
    print("Nesting level 3 " + str(dev[dev['nesting_level'] == 3].shape[0]))
    print("Nesting level 4 " + str(dev[dev['nesting_level'] == 4].shape[0]))
    print("Nesting level 5 " + str(dev[dev['nesting_level'] == 5].shape[0]))
    print("Nesting level 6 " + str(dev[dev['nesting_level'] == 6].shape[0]))

    dev.head()


4
4
3
3
4


In [4]:
import re

def count_graphql_arguments(query: str) -> int:
    # Removing white spaces and escape characters
    query = re.sub(r'\s|\\', '', query)
    
    # Regular expression to find arguments in the query
    pattern = re.compile(r'(\{[^\{]*\})')
    matches = pattern.findall(query)

    # Counting the number of arguments
    count = 0
    for match in matches:
        count += match.count(':')
    return count

### Num_args

In [5]:


#dev['num_args'] = dev['query'].apply(count_query_arguments)

test = False
stats = False

if test:
    query1 = 'query{teacher(where:{_or:[{age:{_eq:\"32\"}},{age:{_eq:\"33\"}}]}){name}}'
    print(count_graphql_arguments(query1))  # Output: 2
    query2 = 'query { templates ( where : { _and : [ { template_type_code : { _neq : "PP" } } , { template_type_code : { _neq : "PPT" } } ] } ) { template_id template_type_code } }'
    print(count_graphql_arguments(query2))  # Output: 2
    query3 = 'query { countrylanguage ( where : { language : { _neq : \"French\" } } , distinct_on : countrycode ) { countrycode } }'
    print(count_graphql_arguments(query3))  # Output: 2
    query4 = 'query { matches_aggregate { aggregate { count } } }'
    print(count_graphql_arguments(query4))  # Output: 0
    query5 = 'query{singer_aggregate(where:{country:{_eq:\"France\"}}){aggregate{avg{age}min{age}max{age}}}}'
    print(count_graphql_arguments(query5))  # Output: 1
    query6 = 'query { tv_channel_aggregate ( where : { language : { _eq : \"English\" } } ) { aggregate { count } } }'
    print(count_graphql_arguments(query6))  # Output: 1
    query7 = '{ conductor ( where : { orchestras : { year_of_founded : { _gt : 2008.0 } } } ) { name } }'
    print(count_graphql_arguments(query7))  # Output: 1
    query8 = '{ flights_aggregate ( where : { sourceairport : { _eq : \"APG\" } } ) { aggregate { count } } }'
    print(count_graphql_arguments(query8))  # Output: 1
    query9 = '{ poker_player_aggregate ( where : { earnings : { _lt : 200000.0 } } ) { aggregate { max { final_table_made } } } }'
    print(count_graphql_arguments(query9))  # Output: 1
    query10 = '{ country_aggregate ( where : { _and : { continent : { _eq : \"Africa\" } , governmentform : { _eq : \"Republic\" } } } ) { aggregate { avg { lifeexpectancy } } } }'
    print(count_graphql_arguments(query10))  # Output: 2
    query11 = '{ matches_aggregate ( where : { _and : { winner_hand : { _eq : \"L\" } , tourney_name : { _eq : \"WTA Championships\" } } } ) { aggregate { count } } }'
    print(count_graphql_arguments(query11))  # Output: 2
    query12 =  '{ paragraphs_aggregate ( where : { document : { document_name : { _eq : \"Summer Show\" } } } ) { aggregate { count } } }'
    print(count_graphql_arguments(query12))  # Output: 1

if stats:
    dev['num_args'].hist()

    print(dev['num_args'].min())
    print(dev['num_args'].max())

    print(dev['num_args'].mean())
    print(dev['num_args'].median())


#### Addding length and schemaId to dataframe

In [37]:
import os
import pandas as pd

def add_schema_info(df):
    schemas_folder = 'SPEGQL-dataset/Schemas'
    schema_folders = sorted(os.listdir(schemas_folder), key=lambda x: os.path.getmtime(os.path.join(schemas_folder, x)))

    for schema_folder in schema_folders:
        if os.path.isdir(os.path.join(schemas_folder, schema_folder)):
            with open(os.path.join(schemas_folder, schema_folder, 'schema.json'), 'r') as schema_file:
                schema_length = sum(1 for line in schema_file)

            new_row = pd.DataFrame({'schemaId': [schema_folder], 'schema_length': [schema_length]})
            df = pd.concat([df, new_row], ignore_index=True)

    return df

# Create an empty DataFrame with the specified columns
data = {'schemaId': [], 'schema_length': []}

# create a dataframe from dev_df_with_results.csv
dev_df = pd.read_csv('dev_df_with_results.csv')

# add schema info to the dataframe
dev_df = add_schema_info(dev_df)



166

### Calculate Schema Complexity

In [39]:
weight_types = 1
weight_fields = 1
weight_input_objects = 1
weight_relationships = 2
weight_arguments = 1

def analyze_schema_complexity(schema_json):
    types_count = 0
    fields_count = 0
    input_objects_count = 0
    relationships_count = 0
    arguments_count = 0

    for type_ in schema_json["__schema"]["types"]:
        types_count += 1

        if type_["kind"] == "INPUT_OBJECT":
            input_objects_count += 1

        if "fields" in type_ and type_["fields"] is not None:
            for field in type_["fields"]:
                fields_count += 1

                if "args" in field:
                    arguments_count += len(field["args"])
                    relationships_count += 1

    complexity_score = (
    (types_count * weight_types)
    + (fields_count * weight_fields)
    + (input_objects_count * weight_input_objects)
    + (relationships_count * weight_relationships)
    + (arguments_count * weight_arguments)
)

    schema_length = len(json.dumps(schema_json))

    return complexity_score, types_count, fields_count, input_objects_count, relationships_count, arguments_count, schema_length

# Load your GraphQL schema JSON file
import json

with open("SPEGQL-dataset/Schemas/activity_1/schema.json", "r") as file:
    schema_json = json.load(file)

complexity_score, types_count, fields_count, input_objects_count, relationships_count, arguments_count, schema_length = analyze_schema_complexity(schema_json)

## Call the function on all schemas and add the counts and complexity score to the DataFrame
for schema_folder in schema_folders:
    if os.path.isdir(os.path.join(schemas_folder, schema_folder)):
        with open(os.path.join(schemas_folder, schema_folder, 'schema.json'), 'r') as schema_file:
            schema_json = json.load(schema_file)
            complexity_score, types_count, fields_count, input_objects_count, relationships_count, arguments_count, schema_length = analyze_schema_complexity(schema_json)
            df.loc[df['schemaId'] == schema_folder, 'schema_total_complexity'] = complexity_score
            df.loc[df['schemaId'] == schema_folder, 'schema_types_count'] = types_count
            df.loc[df['schemaId'] == schema_folder, 'schema_fields_count'] = fields_count
            df.loc[df['schemaId'] == schema_folder, 'schema_input_objects_count'] = input_objects_count
            df.loc[df['schemaId'] == schema_folder, 'schema_relationships_count'] = relationships_count
            df.loc[df['schemaId'] == schema_folder, 'schema_arguments_count'] = arguments_count
df.head()

NameError: name 'schema_folders' is not defined

In [ ]:
## Test 

# Sample GraphQL schema in JSON format
sample_schema = {
    "__schema": {
        "types": [
            {
                "kind": "OBJECT",
                "name": "query_root",
                "fields": [
                    {
                        "name": "author",
                        "args": [
                            {"name": "limit"},
                            {"name": "offset"},
                            {"name": "order_by"},
                            {"name": "where"},
                        ],
                    },
                    {
                        "name": "author_aggregate",
                        "args": [
                            {"name": "distinct_on"},
                            {"name": "limit"},
                            {"name": "offset"},
                            {"name": "order_by"},
                        ],
                    },
                ],
            },
            {
                "kind": "OBJECT",
                "name": "author",
                "fields": [
                    {"name": "id"},
                    {"name": "name"},
                    {"name": "birthdate"},
                ],
            },
            {
                "kind": "INPUT_OBJECT",
                "name": "author_bool_exp",
            },
        ],
    },
}

# Run the analyze_schema_complexity function with the sample schema
complexity_score = analyze_schema_complexity(sample_schema)
print("Complexity score:", complexity_score)

Complexity score: (21, 3, 5, 1, 2, 8, 475)


In [ ]:
import pandas as pd

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping to next line
pd.set_option('display.max_colwidth', None)  # Show the full content of each cell

test_queries = [
'query { hiring { employee_id is_full_time shop_id start_from } }',
'query { teacher ( order_by : { age : asc } ) { name } }',
'query { pets ( where : { pet_age : { _gt : 1 } } ) { petid weight } }',
'query { singer_aggregate { aggregate { count } } }',
'query { singer { birth_year citizenship } }',
'query { ref_template_types { template_type_code template_type_description } }',
'query { airports ( where : { city : { _eq : "Anthony" } } ) { airportcode airportname }}',
'query { singer ( where : { song_name : { _like : "%Hey%" } } ) { name country } }',
'query { players ( order_by : { birth_date : asc } ) { first_name last_name } }',
'query { battle ( where : { bulgarian_commander : { _neq : "Boril" } } ) { name result } }',
'query { stadium ( where : { capacity : { _gte : 5000 , _lte : 10000 } } ) { location name } }',
'query { country_aggregate ( where : { continent : { _eq : "Africa" } } ) { aggregate { count } } }',
'query { airlines_aggregate ( where : { country : { _eq : "USA" } } ) { aggregate { count } } }',
'query { airlines_aggregate ( where : { country : { _eq : "USA" } } ) { aggregate { count } } }',
'query { country_aggregate ( where : { governmentform : { _eq : "Republic" } } ) { aggregate { count } } }',
'query { airlines_aggregate ( where : { _and : { flights : { sourceairport : { _eq : "AHD" } } , airline : { _eq : "United Airlines" } } } ) { aggregate { count } } }',
'query { airlines_aggregate ( where : { _and : { airline : { _eq : "United Airlines" } , flights : { destairport : { _eq : "ASY" } } } } ) { aggregate { count } } }',
'query { paragraphs_aggregate ( where : { document : { document_name : { _eq : "Summer Show" } } } ) { aggregate { count } } }',
'query { country_aggregate ( where : { _and : { countrylanguages : { isofficial : { _eq : "T" } } , name : { _eq : "Afghanistan" } } } ) { aggregate { count } } }',
'query { airlines_aggregate ( where : { _and : { flights : { sourceairport : { _eq : "AHD" } } , airline : { _eq : "United Airlines" } } } ) { aggregate { count } } }'
]

reordered_queries = [
'query { hiring { employee_id shop_id is_full_time start_from } }',
'query { teacher ( order_by : { age : asc } ) { name } }',
'query { pets ( where : { pet_age : { _gt : 1 } } ) { weight petid } }',
'query { singer_aggregate { aggregate { count } } }',
'query { singer { citizenship birth_year } }',
'query { ref_template_types { template_type_description template_type_code } }',
'query { airports ( where : { city : { _eq : "Anthony" } } ) { airportname airportcode }}',
'query { singer ( where : { song_name : { _like : "%Hey%" } } ) { country name } }',
'query { players ( order_by : { birth_date : asc } ) { last_name first_name } }',
'query { battle ( where : { bulgarian_commander : { _neq : "Boril" } } ) { result name } }',
'query { stadium ( where : { capacity : { _gte : 5000 , _lte : 10000 } } ) { name location } }',
'query { country_aggregate ( where : { continent : { _eq : "Africa" } } ) { aggregate { count } } }',
'query { airlines_aggregate ( where : { country : { _eq : "USA" } } ) { aggregate { count } } }',
'query { airlines_aggregate ( where : { country : { _eq : "USA" } } ) { aggregate { count } } }',
'query { country_aggregate ( where : { governmentform : { _eq : "Republic" } } ) { aggregate { count } } }',
'query { airlines_aggregate ( where : { _and : { airline : { _eq : "United Airlines" } , flights : { sourceairport : { _eq : "AHD" } } } } ) { aggregate { count } } }',
'query { airlines_aggregate ( where : { _and : { airline : { _eq : "United Airlines" } , flights : { destairport : { _eq : "ASY" } } } } ) { aggregate { count } } }',
'query { paragraphs_aggregate ( where : { document : { document_name : { _eq : "Summer Show" } } } ) { aggregate { count } } }',
'query { country_aggregate ( where : { _and : { name : { _eq : "Afghanistan" } , countrylanguages : { isofficial : { _eq : "T" } } } } ) { aggregate { count } } }',
'query { airlines_aggregate ( where : { _and : { airline : { _eq : "United Airlines" } , flights : { sourceairport : { _eq : "AHD" } } } } ) { aggregate { count } } }'
]

import pandas as pd

# Create a dictionary with column names as keys and lists as values
data = {
    "queries": test_queries,
    "queries_shifted_order": reordered_queries
}

# Create a pandas DataFrame using the dictionary
df_test_queries = pd.DataFrame(data)

# Display the DataFrame
df_test_queries

## Save df_test_queries to a csv file called semantic_match_test_queries.csv

df_test_queries.to_csv('semantic_match_test_queries.csv', index=False)



### Exploration Of Failed Predictions

In [ ]:


import pandas as pd

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping to next line
pd.set_option('display.max_colwidth', None)  # Show the full content of each cell

df_dev = pd.read_csv('dev_df_with_results.csv')

df_dev = df_dev[df_dev.semantic_match == 0]

df_dev['error_reason'] = ''

df_dev = df_dev[['question', 'query', 'predicted_query', 'error_reason', 'exact_match', 'num_args', 'nesting_level', 'schema_total_complexity']]

df_dev.sample(50).to_csv('qualitative_analysis.csv', index=False)




357

In [1]:
df_dev = pd.read_csv('dev_df_with_results.csv')

# The rows where rows where semantic_match_result is different from exact_match

df_dev[df_dev.semantic_match != df_dev.exact_match]


NameError: name 'pd' is not defined

### Count Query Components Function

In [8]:
import pandas as pd

def count_query_components(df):

    # Define component types
    arguments = ["\\("]
    filters = ["where"]
    query_modifiers = ["order_by", "limit", "distinct_on"]
    logical_operators = ["_and", "_or", "_not"]
    comparison_operators = ["_eq", "_neq", "_lt", "_gt", "_lte", "_gte", "_like"]
    aggregators = ["min", "max", "sum", "avg"]
    orderings = ["asc", "desc"]

    # Create a dictionary of the component types
    component_types = {
        "arguments": arguments,
        "filters": filters,
        "query_modifiers": query_modifiers,
        "logical_operators": logical_operators,
        "comparison_operators": comparison_operators,
        "aggregators": aggregators,
        "orderings": orderings
    }

    # Create an empty dataframe to store the results, initalized with the columns we want
    results_df = pd.DataFrame(columns=["Component_Type", "Component", "Value"])

    # Iterate through the component types and count the number of times each component appears in the query
    for component_type, components in component_types.items(): # .items() returns a tuple of the key and value, in this case the key is the component type and the value is the list of components
        # We initalize components_counts. This is a dictionary where the key is the component and the value is the number of times that component appears in the query
        component_counts = {component: 0 for component in components} 
        total_components = 0

        # Iterate through each component and count the number of times it appears in the query
        for component in components:
            component_count = df['query'].str.count(component).sum() # .str.count() counts the number of times a string appears in a column, we sum this to get the total number of times the component appears in the column
            component_counts[component] = component_count # Add the component count to the component_counts dictionary
            total_components += component_count # Add the component count to the total_components count

            component_percent = (component_count / len(df)) * 100 # Calculate the percentage of queries that contain the component

            new_row = pd.DataFrame({
                "Component_Type": [f"{component_type} relative"],
                "Component": [component],
                "Value": [component_percent]
            })

            results_df = pd.concat([results_df, new_row], ignore_index=True)

        total_percent_components = (total_components / len(df)) * 100

        new_row = pd.DataFrame({
            "Component_Type": [f"{component_type} relative (total)"],
            "Component": [""],
            "Value": [total_percent_components]
        })

        results_df = pd.concat([results_df, new_row], ignore_index=True)

    # Calculate the average nesting level
    avg_nesting = df['query'].apply(nesting_level).mean()

    # Add the average nesting level to the results dataframe
    new_row = pd.DataFrame({
        "Component_Type": ["avg_nesting_level"],
        "Component": [""],
        "Value": [avg_nesting]
    })

    results_df = pd.concat([results_df, new_row], ignore_index=True)

    # Calculate the average number of GraphQL arguments
    avg_args = df['query'].apply(count_graphql_arguments).mean()

    # Add the average number of GraphQL arguments to the results dataframe
    new_row = pd.DataFrame({
        "Component_Type": ["avg_graphql_arguments"],
        "Component": [""],
        "Value": [avg_args]
    })

    # Calculate the length of each query
    df['query_length'] = df['query'].str.len()

    # Calculate max, min and avg query length
    max_query_length = df['query_length'].max()
    min_query_length = df['query_length'].min()
    avg_query_length = df['query_length'].mean()

    # Add max query length to the results dataframe
    new_row = pd.DataFrame({
        "Component_Type": ["max_query_length"],
        "Component": [""],
        "Value": [max_query_length]
    })
    results_df = pd.concat([results_df, new_row], ignore_index=True)

    # Add min query length to the results dataframe
    new_row = pd.DataFrame({
        "Component_Type": ["min_query_length"],
        "Component": [""],
        "Value": [min_query_length]
    })
    results_df = pd.concat([results_df, new_row], ignore_index=True)

    # Add avg query length to the results dataframe
    new_row = pd.DataFrame({
        "Component_Type": ["avg_query_length"],
        "Component": [""],
        "Value": [avg_query_length]
    })

    results_df = pd.concat([results_df, new_row], ignore_index=True)

    if 'question_length' in df.columns:
        # Calculate max, min and avg question length
        max_question_length = df['question_length'].max()
        min_question_length = df['question_length'].min()
        avg_question_length = df['question_length'].mean()

        # Add max question length to the results dataframe
        new_row = pd.DataFrame({
            "Component_Type": ["max_question_length"],
            "Component": [""],
            "Value": [max_question_length]
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)

        # Add min question length to the results dataframe
        new_row = pd.DataFrame({
            "Component_Type": ["min_question_length"],
            "Component": [""],
            "Value": [min_question_length]
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)

        # Add avg question length to the results dataframe
        new_row = pd.DataFrame({
            "Component_Type": ["avg_question_length"],
            "Component": [""],
            "Value": [avg_question_length]
        })

    results_df = pd.concat([results_df, new_row], ignore_index=True)

    return results_df
